﻿# 通过 PyTorch 构建神经网络

深度学习网络一般量级都很大，包含数百个层级，这也是为什么叫“深度”学习网络。你可以像在上个 notebook 展示的一样，仅使用权重矩阵构建深度网络，但是这通常很繁琐并且不好实施。PyTorch 有一个很方便的模块 `nn`，可以有效地构建大型神经网络。

In [ ]:
# Import necessary packages

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch

import helper

import matplotlib.pyplot as plt

现在我们要构建一个大型网络，并用它解决识别图像中的文字的这一难题。我们将使用 MNIST 数据集，这个数据集由灰色的手写数字组成。每个图像都是 28x28，如以下示例所示：

<img src='assets/mnist.png'>

我们的目标是构建一个神经网络，可以预测图像中的数字。

首先，我们需要获取数据集。这些数据位于 `torchvision` 软件包中。以下代码将下载 MNIST 数据集，然后为我们创建训练数据集和测试数据集。不用太在意细节内容，稍后会详细学习的。

In [ ]:
### Run this cell

from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

我们将训练数据加载到 `trainloader` 中，并使用 `iter(trainloader)` 使其变成迭代器。之后，我们将用它循环访问数据集以进行训练，例如

In [ ]:
for image, label in trainloader:
    ## do things with images and labels

我在创建 `trainloader` 时，将批次大小设为 64，并设置为 `shuffle=True`。批次大小是指我们在一次迭代中从数据加载器获取并经过网络的图像数量。`shuffle=True` 表示每次重新访问数据加载器时，随机重排数据集。但是现在我仅获取第一批数据，以便查看数据。从下方可以看出，`images` 是一个张量，大小为 `(64, 1, 28, 28)`。因此，每批有 64 个图像，图像有 1 个颜色通道，共有 28x28 个图像。

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

In [ ]:
<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64])


下面是一个图像示例。

In [ ]:
plt.imshow(images[1].numpy().squeeze(), cmap='Greys_r');

![png](output_7_0.png)


首先，我们要使用权重矩阵和矩阵乘法，用此数据集构建一个简单的网络。然后，我们将学习如何使用 PyTorch 的 `nn` 模块构建该网络。

神经网络又称为*全连接*或*密集*网络。一个层级中的每个单元都与下个层级中的每个单元相连。在全连接网络中，每个层级的输入必须是一维向量（可以作为一批样本堆叠为二维张量）。但是，我们的图像是 28x28 二维张量，因此我们需要将其转换为一维向量。考虑到大小问题，我们需要将形状为 `(64, 1, 28, 28)` 的批次图像变形为 `(64, 784)`，784 等于 28 x 28。这一步通常称为*扁平化*，我们将二维图像扁平化为一维向量。

之前，我们试过了构建具有一个输出单元的简单网络。现在，我们需要 10 个输出单元，每个数字对应一个单元。如果要预测出图像中显示的数字，我们必须计算该图像属于任何数字或类别的概率。我们会得到一个离散概率分布，告诉我们图像最有可能属于哪个类别。这就是说，我们需要 10 个输出单元，对应 10 个类别（数字）。下面讲解如何将网络输出转换为概率分布。

> **练习：**将 `images`扁平化。然后构建一个多层网络，有 784 个输入单元、256 个隐藏单元和 10 个输出单元，并对权重和偏差使用随机张量。目前，我们对隐藏层使用 S 型激活函数。输出层暂时不需要激活函数，下一步我们将添加计算概率分布的激活函数。

In [ ]:
## Solution
def activation(x):
    return 1/(1+torch.exp(-x))

# Flatten the input images
inputs = images.view(images.shape[0], -1)

# Create parameters
w1 = torch.randn(784, 256)
b1 = torch.randn(256)

w2 = torch.randn(256, 10)
b2 = torch.randn(10)

h = activation(torch.mm(inputs, w1) + b1)

out = torch.mm(h, w2) + b2

现在网络有 10 个输出了。我们向网络中传入一个图像，并获得类别概率分布，告诉我们图像最有可能属于哪个数字/类别。结果如下所示：
<img src='assets/image_distribution.png' width=500px>

可以看出每个类别的概率大致相等。这是未训练网络的结果，网络尚未见过任何数据，因此返回均匀分布，每个类别的概率相等。

可以用 [**softmax** 函数]计算概率分布(https://en.wikipedia.org/wiki/Softmax_function)。数学公式为：

$$
\Large \sigma(x_i) = \cfrac{e^{x_i}}{\sum_k^K{e^{x_k}}}
$$

它会将每个输入 $x_i$ 都缩放到 0 和 1 之间并标准化值，得出标准概率分布，其中概率总和是 1。

> **练习：**实现一个进行 softmax 运算的函数 `softmax` ，并针对批次中的每个样本返回概率分布。注意，在运算时需要注意形状。如果有一个张量 `a` 的形状为 `(64, 10)`，另一个张量 `b` 的形状为 `(64,)`，进行 `a/b` 运算将出错，因为 PyTorch 会对列进行除法运算（称为广播），但是大小不匹配。提示：对于 64 个样本中的每个样本，你可以除以一个值，即分母中的和。因此需要使 `b` 变形为 `(64, 1)`。这样的话，PyTorch 将使 `a` 中每行的10 个值除以 `b` 中每行的一个值。另外，要注意求和的方式。你要在 `torch.sum` 中定义 `dim` 。`dim=0` 会对行求和，而 `dim=1` 会对列求和。

In [ ]:
## Solution
def softmax(x):
    return torch.exp(x)/torch.sum(torch.exp(x), dim=1).view(-1, 1)

probabilities = softmax(out)

# Does it have the right shape? Should be (64, 10)
print(probabilities.shape)
# Does it sum to 1?
print(probabilities.sum(dim=1))

In [ ]:
torch.Size([64, 10])
tensor([ 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000])


## 通过 PyTorch 构建网络

PyTorch 提供了`nn`模块，大大地简化了网络构建过程。我将演示如何构建上述同一个网络，即包含 784 个输入、256 个隐藏单元、10 个输出单元和一个 softmax 输出。

In [ ]:
from torch import nn

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Inputs to hidden layer linear transformation
        self.hidden = nn.Linear(784, 256)
        # Output layer, 10 units - one for each digit
        self.output = nn.Linear(256, 10)
        
        # Define sigmoid activation and softmax output 
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.output(x)
        x = self.softmax(x)
        
        return x

分步讲解。

In [ ]:
class Network(nn.Module):

先继承 `nn.Module`。与 `super().__init__()` 相结合，创建一个跟踪架构的类，并提供大量有用的方法和属性。注意，在为网络创建类时，必须继承 `nn.Module`。类可以随意命名。

In [ ]:
self.hidden = nn.Linear(784, 256)

这行创建一个线性转换模块 $x\mathbf{W} + b$，其中有 784 个输入和 256 个输出，并赋值给 `self.hidden`。该模块会自动创建权重和偏差张量，供我们在 `forward` 方法中使用。创建网络 (`net`) 后，你可以使用 `net.hidden.weight` 和 `net.hidden.bias` 访问权重和偏差张量。

In [ ]:
self.output = nn.Linear(256, 10)

同样，这里会创建另一个有 256 个输入和 10 个输出的线性转换。

In [ ]:
self.sigmoid = nn.Sigmoid()
self.softmax = nn.Softmax(dim=1)

然后，我定义了 S 型激活函数和 softmax 输出的运算。在 `nn.Softmax(dim=1)` 中设置 `dim=1` 会计算各个列的 softmax 值。

In [ ]:
def forward(self, x):

用 `nn.Module` 创建的 PyTorch 网络必须定义 `forward` 方法。它会接受一个张量 `x` 并将其传入你在 `__init__` 方法中定义的运算。

In [ ]:
x = self.hidden(x)
x = self.sigmoid(x)
x = self.output(x)
x = self.softmax(x)

我们将输入张量 `x` 传入重新赋值给 `x` 的每个运算。可以看出输入张量经过隐藏层，然后经过 S 型函数、输出层，最终经过 softmax 函数。.变量可以命名为任何名称，只要运算的输入和输出与你要构建的网络架构匹配即可。你在 `__init__` 方法中的定义顺序不重要，但是需要在 `forward` 方法中正确地设定运算顺序。

现在我们可以创建一个 `Network` 对象。

In [ ]:
# Create the network and look at it's text representation
model = Network()
model

In [ ]:
Network(
  (hidden):Linear(in_features=784, out_features=256, bias=True)
  (output):Linear(in_features=256, out_features=10, bias=True)
  (sigmoid):Sigmoid()
  (softmax):Softmax()
)



你可以使用 `torch.nn.functional` 模块来更简练清晰地定义网络。这是最常见的网络定义方式，因为很多运算是简单的元素级函数。我们通常将此模块导入为 `F`，即 `import torch.nn.functional as F`。

In [ ]:
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # Inputs to hidden layer linear transformation
        self.hidden = nn.Linear(784, 256)
        # Output layer, 10 units - one for each digit
        self.output = nn.Linear(256, 10)
        
    def forward(self, x):
        # Hidden layer with sigmoid activation
        x = F.sigmoid(self.hidden(x))
        # Output layer with softmax activation
        x = F.softmax(self.output(x), dim=1)
        
        return x

### 激活函数

到目前为止，我们只学习了 softmax 激活函数，但是通常任何函数都可以用作激活函数。但是，要使网络能逼近非线性函数，激活函数必须是非线性函数。下面是一些常见的激活函数示例：Tanh（双曲正切）和 ReLU（修正线性单元）。

<img src="assets/activation.png" width=700px>

在实践中，ReLU 几乎一直用作隐藏层激活函数。

### 构建神经网络

<img src="assets/mlp_mnist.png" width=600px>

> **练习：**请创建如下网络：输入层有 784 个单元，然后是有 128 个单元的隐藏层和一个 ReLU 激活函数，接着是有 64 个单元的隐藏层和一个 ReLU 激活函数，最终是一个应用 softmax 激活函数的输出层（如上所示）。你可以通过 `nn.ReLU` 模块或 `F.relu` 函数应用 ReLU 激活函数。

In [ ]:
## Solution

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # Defining the layers, 128, 64, 10 units each
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        # Output layer, 10 units - one for each digit
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        ''' Forward pass through the network, returns the output logits '''
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        
        return x

model = Network()
model

In [ ]:
Network(
  (fc1):Linear(in_features=784, out_features=128, bias=True)
  (fc2):Linear(in_features=128, out_features=64, bias=True)
  (fc3):Linear(in_features=64, out_features=10, bias=True)
)



### 初始化权重和偏差

权重和偏差会自动初始化，但是你可以自定义它们的初始化方式。权重和偏差是附加到所定义层级上的张量，例如，你可以通过 `model.fc1.weight` 获取权重。

In [ ]:
print(model.fc1.weight)
print(model.fc1.bias)

In [ ]:
Parameter containing:
tensor([[-2.3278e-02, -1.2170e-03, -1.1882e-02, ..., 3.3567e-02,
          4.4827e-03, 1.4840e-02],
        [ 4.8464e-03, 1.9844e-02, 3.9791e-03, ..., -2.6048e-02,
         -3.5558e-02, -2.2386e-02],
        [-1.9664e-02, 8.1722e-03, 2.6729e-02, ..., -1.5122e-02,
          2.7632e-02, -1.9567e-02],
        ...,
        [-3.3571e-02, -2.9686e-02, -2.1387e-02, ..., 3.0770e-02,
          1.0800e-02, -6.5941e-03],
        [ 2.9749e-02, 1.2849e-02, 2.7320e-02, ..., -1.9899e-02,
          2.7131e-02, 2.2082e-02],
        [ 1.3992e-02, -2.1520e-02, 3.1907e-02, ..., 2.2435e-02,
          1.1370e-02, 2.1568e-02]])
Parameter containing:
tensor(1.00000e-02 *
       [-1.3222, 2.4094, -2.1571, 3.2237, 2.5302, -1.1515, 2.6382,
        -2.3426, -3.5689, -1.0724, -2.8842, -2.9667, -0.5022, 1.1381,
         1.2849, 3.0731, -2.0207, -2.3282, 0.3168, -2.8098, -1.0740,
        -1.8273, 1.8692, 2.9404, 0.1783, 0.9391, -0.7085, -1.2522,
        -2.7769, 0.0916, -1.4283, -0.3267, -1.6876, -1.8580, -2.8724,
        -3.5512, 3.2155, 1.5532, 0.8836, -1.2911, 1.5735, -3.0478,
        -1.3089, -2.2117, 1.5162, -0.8055, -1.3307, -2.4267, -1.2665,
         0.8666, -2.2325, -0.4797, -0.5448, -0.6612, -0.6022, 2.6399,
         1.4673, -1.5417, -2.9492, -2.7507, 0.6157, -0.0681, -0.8171,
        -0.3554, -0.8225, 3.3906, 3.3509, -1.4484, 3.5124, -2.6519,
         0.9721, -2.5068, -3.4962, 3.4743, 1.1525, -2.7555, -3.1673,
         2.2906, 2.5914, 1.5992, -1.2859, -0.5682, 2.1488, -2.0631,
         2.6281, -2.4639, 2.2622, 2.3632, -0.1979, 0.7160, 1.7594,
         0.0761, -2.8886, -3.5467, 2.7691, 0.8280, -2.2398, -1.4602,
        -1.3475, -1.4738, 0.6338, 3.2811, -3.0628, 2.7044, 1.2775,
         2.8856, -3.3938, 2.7056, 0.5826, -0.6286, 1.2381, 0.7316,
        -2.4725, -1.2958, -3.1543, -0.8584, 0.5517, 2.8176, 0.0947,
        -1.6849, -1.4968, 3.1039, 1.7680, 1.1803, -1.4402, 2.5710,
        -3.3057, 1.9027])


要自定义初始化过程，我们需要原地修改这些张量。这些实际上是 autograd *变量*，因此我们需要使用 `model.fc1.weight.data` 获取实际张量。获得张量后，我们可以用 0（对于偏差）或随机正常值填充这些张量。

In [ ]:
# Set biases to all zeros
model.fc1.bias.data.fill_(0)

In [ ]:
tensor([ 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.])




In [ ]:
# sample from random normal with standard dev = 0.01
model.fc1.weight.data.normal_(std=0.01)

In [ ]:
tensor([[ 5.9294e-03, 9.6176e-04, 9.5034e-03, ..., -1.2588e-03,
          3.4878e-03, 1.4495e-02],
        [-9.3127e-04, -1.0287e-02, 3.5423e-03, ..., 1.1155e-02,
          2.5012e-03, 1.3503e-02],
        [-1.5078e-02, 8.1119e-03, 5.0678e-03, ..., 2.8613e-03,
          9.7443e-03, -6.5530e-03],
        ...,
        [-2.2938e-03, -1.8630e-02, -4.2459e-03, ..., 1.4718e-02,
          2.7655e-03, 3.2661e-03],
        [-3.0470e-03, 1.2617e-02, -7.9772e-03, ..., -1.4699e-02,
         -9.1425e-03, 5.3492e-03],
        [ 2.5911e-03, 2.4797e-03, -6.1172e-03, ..., 1.0584e-02,
         -1.0307e-02, -3.1901e-03]])



### 前向传递

我们已经创建好网络，看看传入图像后会发生什么。

In [ ]:
# Grab some data 
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Resize images into a 1D vector, new shape is (batch size, color channels, image pixels) 
images.resize_(64, 1, 784)
# or images.resize_(images.shape[0], 1, 784) to automatically get batch size

# Forward pass through the network
img_idx = 0
ps = model.forward(images[img_idx,:])

img = images[img_idx]
helper.view_classify(img.view(1, 28, 28), ps)

![png](output_28_0.png)


是的，我们的网络还不能判断出这个数字。这是因为我们尚未训练它，所有权重都是随机的！

### 使用 `nn.Sequential`

PyTorch 提供了一种方便的方法来构建这类网络（其中张量按顺序执行各种运算）：`nn.Sequential` ([文档](https://pytorch.org/docs/master/nn.html#torch.nn.Sequential))。使用它来构建等效网络：

In [ ]:
# Hyperparameters for our network
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

# Build a feed-forward network
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.Softmax(dim=1))
print(model)

# Forward pass through the network and display output
images, labels = next(iter(trainloader))
images.resize_(images.shape[0], 1, 784)
ps = model.forward(images[0,:])
helper.view_classify(images[0].view(1, 28, 28), ps)

In [ ]:
Sequential(
  (0):Linear(in_features=784, out_features=128, bias=True)
  (1):ReLU()
  (2):Linear(in_features=128, out_features=64, bias=True)
  (3):ReLU()
  (4):Linear(in_features=64, out_features=10, bias=True)
  (5):Softmax()
)



![png](output_30_1.png)


通过传入相应的索引即可执行运算。例如，如果你想获得第一个线性运算并查看权重，可以使用 `model[0]`。

In [ ]:
print(model[0])
model[0].weight

In [ ]:
Linear(in_features=784, out_features=128, bias=True)





Parameter containing:
tensor([[-7.0372e-03, 2.9689e-02, 2.6028e-02, ..., 1.1196e-02,
         -2.1928e-02, -3.4886e-02],
        [ 1.0624e-04, -1.6610e-02, -2.2891e-02, ..., -6.4412e-03,
         -2.5026e-02, 1.0674e-02],
        [-2.4707e-02, 1.4146e-02, 2.0084e-02, ..., 1.2227e-02,
          2.3441e-02, -9.5175e-03],
        ...,
        [-2.0119e-02, 7.2614e-03, -1.3481e-02, ..., 8.1745e-03,
         -1.3348e-02, -1.1955e-02],
        [ 3.2282e-02, 1.8674e-02, -3.0826e-02, ..., 1.7296e-02,
         -2.6710e-02, -3.0684e-02],
        [ 1.7686e-02, 3.1376e-02, -2.2645e-02, ..., 2.4630e-02,
          3.1129e-02, 7.0939e-03]])



还可以传入 `OrderedDict` 以命名单个层级和运算，而不是使用递增的整数。注意，因为字典键必须是唯一的，所以_每个运算都必须具有不同的名称_。

In [ ]:
from collections import OrderedDict
model = nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                      ('relu1', nn.ReLU()),
                      ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                      ('relu2', nn.ReLU()),
                      ('output', nn.Linear(hidden_sizes[1], output_size)),
                      ('softmax', nn.Softmax(dim=1))]))
model

In [ ]:
Sequential(
  (fc1):Linear(in_features=784, out_features=128, bias=True)
  (relu1):ReLU()
  (fc2):Linear(in_features=128, out_features=64, bias=True)
  (relu2):ReLU()
  (output):Linear(in_features=64, out_features=10, bias=True)
  (softmax):Softmax()
)



现在你可以通过整数或名称访问层级了

In [ ]:
print(model[0])
print(model.fc1)

In [ ]:
Linear(in_features=784, out_features=128, bias=True)
Linear(in_features=784, out_features=128, bias=True)


在下个 notebook 中，我们将学习如何训练神经网络，以便准确预测 MNIST 图像中出现的数字。